## IMPORTING THE REQUIRED LIBRARIES AND DATA

In [3]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

# Display all columns in one line
pd.set_option('display.max_columns', None)   # show all columns
pd.set_option('display.width', 1000)         # set a wide width to avoid wrapping


# Load CSV
df_csv = pd.read_csv('tn.movie_budgets.csv\\tn.movie_budgets.csv')

# Load SQLite
conn = sqlite3.connect('im.db\im.db')


In [4]:
# Check CSV columns and first 5 rows
print("CSV columns:")
print(df_csv.columns.tolist())

print("\nCSV sample data:")
print(df_csv.head())

CSV columns:
['id', 'release_date', 'movie', 'production_budget', 'domestic_gross', 'worldwide_gross']

CSV sample data:
   id  release_date                                        movie production_budget domestic_gross worldwide_gross
0   1  Dec 18, 2009                                       Avatar      $425,000,000   $760,507,625  $2,776,345,279
1   2  May 20, 2011  Pirates of the Caribbean: On Stranger Tides      $410,600,000   $241,063,875  $1,045,663,875
2   3   Jun 7, 2019                                 Dark Phoenix      $350,000,000    $42,762,350    $149,762,350
3   4   May 1, 2015                      Avengers: Age of Ultron      $330,600,000   $459,005,868  $1,403,013,963
4   5  Dec 15, 2017            Star Wars Ep. VIII: The Last Jedi      $317,000,000   $620,181,382  $1,316,721,747


In [5]:
# Check available tables
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("SQLite tables:")
print(tables)
 

SQLite tables:
            name
0   movie_basics
1      directors
2      known_for
3     movie_akas
4  movie_ratings
5        persons
6     principals
7        writers


In [6]:
# Inspect movie_basics table
df_movie_basics = pd.read_sql_query("SELECT * FROM movie_basics;", conn)

print("\nFirst 5 rows of movie_basics:")
print(df_movie_basics)

print("\nColumns in movie_basics:")
print(df_movie_basics.columns.tolist())



First 5 rows of movie_basics:
         movie_id                                primary_title                               original_title  start_year  runtime_minutes                genres
0       tt0063540                                    Sunghursh                                    Sunghursh        2013            175.0    Action,Crime,Drama
1       tt0066787              One Day Before the Rainy Season                              Ashad Ka Ek Din        2019            114.0       Biography,Drama
2       tt0069049                   The Other Side of the Wind                   The Other Side of the Wind        2018            122.0                 Drama
3       tt0069204                              Sabse Bada Sukh                              Sabse Bada Sukh        2018              NaN          Comedy,Drama
4       tt0100275                     The Wandering Soap Opera                        La Telenovela Errante        2017             80.0  Comedy,Drama,Fantasy
...           .

## DATA CLEANING

In [7]:
# Check missing values
print("Missing values in CSV:")
print(df_csv.isnull().sum())

print("\nMissing values in movie_basics:")
print(df_movie_basics.isnull().sum())

# Check duplicates
print("\nDuplicate movie titles in CSV:")
print(df_csv['movie'].duplicated().sum())

print("\nDuplicate primary_title in movie_basics:")
print(df_movie_basics['primary_title'].duplicated().sum())

# Clean numeric columns
df_csv['production_budget'] = df_csv['production_budget'].replace('[\$,]', '', regex=True)
df_csv['production_budget'] = pd.to_numeric(df_csv['production_budget'], errors='coerce')

df_csv['worldwide_gross'] = df_csv['worldwide_gross'].replace('[\$,]', '', regex=True)
df_csv['worldwide_gross'] = pd.to_numeric(df_csv['worldwide_gross'], errors='coerce')

# Clean movie titles: strip spaces, lowercase, remove punctuation
df_csv['movie_clean'] = ( df_csv['movie'].str.strip().str.lower().str.replace(r"[^\w\s]", "", regex=True))

df_movie_basics['primary_title_clean'] = (df_movie_basics['primary_title'].str.strip().str.lower().str.replace(r"[^\w\s]", "", regex=True))


Missing values in CSV:
id                   0
release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
dtype: int64

Missing values in movie_basics:
movie_id               0
primary_title          0
original_title        21
start_year             0
runtime_minutes    31739
genres              5408
dtype: int64

Duplicate movie titles in CSV:
84

Duplicate primary_title in movie_basics:
10073


In [8]:
# Merge on primary_title_clean
df_merge_primary = pd.merge(
    df_csv,
    df_movie_basics[['movie_id', 'primary_title', 'genres', 'start_year', 'primary_title_clean']],
    left_on='movie_clean',
    right_on='primary_title_clean',
    how='inner'
)


In [9]:
# Load movie_akas
df_movie_akas = pd.read_sql_query("SELECT movie_id, title FROM movie_akas", conn)

In [10]:
# Clean movie_id type
df_movie_basics['movie_id'] = df_movie_basics['movie_id'].astype(str)
df_movie_akas['movie_id'] = df_movie_akas['movie_id'].astype(str)

# Clean movie_akas titles
df_movie_akas['clean_aka_title'] = (df_movie_akas['title'].str.strip().str.lower().str.replace(r"[^\w\s]", "", regex=True))


In [11]:
# Find unmatched movies after primary_title merge
matched_movie_titles = df_merge_primary['movie'].unique()
df_csv_unmatched = df_csv[~df_csv['movie'].isin(matched_movie_titles)]

print(f"Number of unmatched movies after primary_title merge: {df_csv_unmatched.shape[0]}")

Number of unmatched movies after primary_title merge: 3309


In [12]:
# Merge unmatched movies with AKAS
df_merge_akas = pd.merge(
    df_csv_unmatched,
    df_movie_akas[['movie_id', 'clean_aka_title']],
    left_on='movie_clean',
    right_on='clean_aka_title',
    how='inner'
)

# Bring in genres and start_year
df_merge_akas_full = pd.merge(
    df_merge_akas,
    df_movie_basics[['movie_id', 'genres', 'start_year']],
    on='movie_id',
    how='left'
)

# Concatenate both DataFrames
df_merged_combined = pd.concat([df_merge_primary, df_merge_akas_full], ignore_index=True)

# Drop duplicates based on 'movie'
df_merged_combined = df_merged_combined.drop_duplicates(subset='movie', keep='first')

# Show final merged result
print("\nFINAL Merged DataFrame shape:", df_merged_combined.shape)
print("\nSample merged data with genres:")
print(df_merged_combined[['movie', 'genres', 'production_budget', 'worldwide_gross']].head(5))

# --- Do not re-merge on raw movie and primary_title here ---
# That part of your code was redundant and would undo the work.

# Optional: Print unmatched movie titles again if you want:
csv_titles = set(df_csv['movie_clean'].unique())
primary_titles = set(df_movie_basics['primary_title_clean'].unique())
aka_titles = set(df_movie_akas['clean_aka_title'].unique())

total_db_titles = primary_titles.union(aka_titles)

unmatched_movies = csv_titles - total_db_titles
print(f"\nNumber of completely unmatched movies: {len(unmatched_movies)}")
if len(unmatched_movies) > 0:
    print("Example unmatched movies:", list(unmatched_movies)[:10])

# Done!



FINAL Merged DataFrame shape: (2595, 13)

Sample merged data with genres:
                                         movie                    genres  production_budget  worldwide_gross
0                                       Avatar                    Horror          425000000       2776345279
1  Pirates of the Caribbean: On Stranger Tides  Action,Adventure,Fantasy          410600000       1045663875
2                                 Dark Phoenix   Action,Adventure,Sci-Fi          350000000        149762350
3                      Avengers: Age of Ultron   Action,Adventure,Sci-Fi          330600000       1403013963
4                       Avengers: Infinity War   Action,Adventure,Sci-Fi          300000000       2048134200

Number of completely unmatched movies: 3103
Example unmatched movies: ['black water transit', 'bend it like beckham', 'la guerre du feu', 'my cousin vinny', 'the thousand miles', 'seed of chucky', 'men women and children', 'artie langes beer league', 'osama', 'urbania']

## DATA CLEANING


In [13]:
# Check missing values again in merged data
print(df_merged_combined[['genres', 'production_budget', 'worldwide_gross']].isnull().sum())

# Check how many genres are 'missing'
missing_genres_count = df_merged_combined['genres'].isnull().sum()
print(f"Movies with missing genres: {missing_genres_count}")

# Check how many budgets/grosses are 0 or very small
print("\nMovies with 0 or missing production_budget:")
print(df_merged_combined[df_merged_combined['production_budget'] <= 0][['movie', 'production_budget']])

print("\nMovies with 0 or missing worldwide_gross:")
print(df_merged_combined[df_merged_combined['worldwide_gross'] <= 0][['movie', 'worldwide_gross']])


genres               26
production_budget     0
worldwide_gross       0
dtype: int64
Movies with missing genres: 26

Movies with 0 or missing production_budget:
Empty DataFrame
Columns: [movie, production_budget]
Index: []

Movies with 0 or missing worldwide_gross:
                   movie  worldwide_gross
487               Bright                0
516     Call of the Wild                0
754     The Ridiculous 6                0
755               Midway                0
912   The Rhythm Section                0
...                  ...              ...
4318  The Horror Network                0
4319   Le bonheur d'Elza                0
4329    El rey de Najayo                0
4331    Brooklyn Bizarre                0
4334           Arrowhead                0

[240 rows x 2 columns]
